# Match state
Trying to generate a way to represent the current state of the game in a vector that can be plugged into modeling methods.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import leaguedata as ld

#Progress bars wooo
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Markdown

In [63]:
matches = ld.get_all_matches()
print(len(matches))


def get_match_state(match_num, min_num):
    match_state = []
    
    match = matches[match_num]
    
    gold_types = ['goldblue', 'goldred', 'goldblueTop', 
                  'goldblueJungle', 'goldblueMiddle', 'goldblueADC', 
                  'goldblueSupport', 'goldredTop', 'goldredJungle',
                  'goldredMiddle', 'goldredADC', 'goldredSupport']
    
    for gold_type in gold_types:
        gold = match.gold_type(gold_type)[min_num]
        match_state.append(gold)
        

    bkills = matches[match_num].blue_kills().values
    num_bkills = 0
    for kill in bkills:
        if(kill[2] < min_num):
            num_bkills +=1
            
    match_state.append(num_bkills)
    
    rkills = matches[match_num].red_kills().values
    num_rkills = 0
    for kill in rkills:
        if(kill[2] < min_num):
            num_rkills +=1
            
    match_state.append(num_rkills)
    
    bturrets = matches[match_num].blue_structures().values
    num_bturrets = 0
    num_binhibs = 0
    for turret in bturrets:
        if(turret[2] < min_num and turret[1] == 'bTowers'):
            num_bturrets +=1
        if(turret[2] < min_num and turret[1] == 'bInhibs'):
            num_binhibs +=1
            
    match_state.append(num_bturrets)
    match_state.append(num_binhibs)
    
    return match_state

7620


In [64]:
get_match_state(99, 35)

[55837.0,
 59648.0,
 12599.0,
 9010.0,
 13082.0,
 13769.0,
 7377.0,
 11749.0,
 11088.0,
 12570.0,
 15257.0,
 8984.0,
 11,
 20,
 5,
 0]

In [15]:
matches[0].gold_type('goldblue')[2]

2711

In [55]:
matches[0].blue_structures()

,Address,Team,Time,Lane,Type
0,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,27.542,MID_LANE,BASE_TURRET
1,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,39.269,MID_LANE,NEXUS_TURRET
2,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,33.583,BOT_LANE,INNER_TURRET
3,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,23.038,TOP_LANE,INNER_TURRET
4,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,16.556,BOT_LANE,OUTER_TURRET
5,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,15.014,MID_LANE,OUTER_TURRET
6,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,39.376,MID_LANE,NEXUS_TURRET
7,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,15.217,MID_LANE,INNER_TURRET
8,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,17.856,TOP_LANE,OUTER_TURRET
97382,http://matchhistory.na.leagueoflegends.com/en/...,bInhibs,36.686,MID_LANE,INHIBITOR
